## Loading the Train and Test

In [45]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras 
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense

In [46]:
train  = pd.read_csv("customer_churn_dataset-training-master.csv")
test  = pd.read_csv("customer_churn_dataset-testing-master.csv")

In [47]:
train.dropna(inplace=True)
test.dropna(inplace = True)

In [48]:
train.drop(columns = ['CustomerID'], inplace =True)
test.drop(columns = ['CustomerID'], inplace =True)

In [49]:
train

,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.00,17.0,1.0
1,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.00,6.0,1.0
2,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.00,3.0,1.0
3,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.00,29.0,1.0
4,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.00,20.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
440828,42.0,Male,54.0,15.0,1.0,3.0,Premium,Annual,716.38,8.0,0.0
440829,25.0,Female,8.0,13.0,1.0,20.0,Premium,Annual,745.38,2.0,0.0
440830,26.0,Male,35.0,27.0,1.0,5.0,Standard,Quarterly,977.31,9.0,0.0
440831,28.0,Male,55.0,14.0,2.0,0.0,Standard,Quarterly,602.55,2.0,0.0


In [50]:
train = pd.get_dummies(train,columns= ["Gender", "Subscription Type", "Contract Length"], drop_first=True,  dtype = int)
test = pd.get_dummies(test,columns= ["Gender", "Subscription Type", "Contract Length"], drop_first =True, dtype = int)

In [51]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train)
X_test_scaled = scaler.transform(test)

In [52]:
X_train = train.iloc[:,:-1]
X_test = test.iloc[:,:-1]
y_train = train.iloc[:,-1]
y_test = test.iloc[:,-1]

In [53]:
X_train.shape[1]

12

## Building the Model

In [59]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import  Adam
from tensorflow.keras.layers import BatchNormalization, Activation
import keras_tuner as kt
callback = EarlyStopping( 
    min_delta = 0.001,
    monitor = "val_loss",
    patience = 3,
    mode = 'min',
    restore_best_weights = True,
    verbose = 1,
    baseline = None
)
def build_model(hp):
    model = Sequential()
    model.add(Dense(128,name= 'Layer1',kernel_initializer = HeNormal(), input_shape =(X_train.shape[1],) ))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(128, name ='layer2', ))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(1, activation = "sigmoid", name ='output'))
    #opti = Adam(learning_rate=0.0001, clipvalue = 1.0)
    opt = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])
    model.compile(loss = "binary_crossentropy", optimizer = opt, metrics = ["accuracy"])
    return model
tuner = kt.RandomSearch(
    build_model,
    objective = "val_accuracy",
    max_trials=5,
    directory = "customer-hp",
    project_name = 'optimiser_tuning',
    overwrite = True
)
tuner.search(X_train,y_train, validation_split=0.2,epochs=2, verbose =1, callbacks=[callback])

Trial 4 Complete [00h 01m 16s]
val_accuracy: 0.49800947308540344

Best val_accuracy So Far: 0.5005273818969727
Total elapsed time: 00h 07m 39s
